In [ ]:
%pip install streamlit

In [ ]:

import requests
import logging
from typing import Optional
import streamlit as st

URL="https://unstats.un.org/SDGAPI/v1/sdg/Indicator/List"
TIMEOUT = 10  # seconds

UN_API_BASE = URL  # Base URL for UN SDG API

def fetch_indicators() -> Optional[dict]:
    """Pulls SDG indicators from the UN API."""
    endpoint = f"{UN_API_BASE}/Indicator/List"
    try:
        response = requests.get(endpoint, timeout=TIMEOUT)
        response.raise_for_status()
        return response.json()
    except requests.RequestException as e:
        logging.error(f"Error fetching indicators: {e}")
        return None

def fetch_indicator_data(indicator_code: str) -> Optional[dict]:
    """
    Fetch detailed data for a specific UN indicator.
    
    Args:
        indicator_code (str): e.g., "1.1.1"
    
    Returns:
        dict or None
    """
    endpoint = f"{UN_API_BASE}/Indicator/{indicator_code}/Data"
    try:
        response = requests.get(endpoint, timeout=TIMEOUT)
        response.raise_for_status()
        return response.json()
    except requests.RequestException as e:
        logging.error(f"Error fetching data for {indicator_code}: {e}")
        return None

def fetch_goals() -> Optional[dict]:
    """Fetches all SDG goals and metadata."""
    endpoint = f"{UN_API_BASE}/Goal/List"
    try:
        response = requests.get(endpoint, timeout=TIMEOUT)
        response.raise_for_status()
        return response.json()
    except requests.RequestException as e:
        logging.error(f"Error fetching SDG goals: {e}")
        return None
    
    # tests/test_api_fetch.py
# from src.python.api_fetch import fetch_indicators, fetch_indicator_data, fetch_goals

def test_all():
    print("\n🔎 Testing UN SDG API connectivity...")
    
    indicators = fetch_indicators()
    if indicators:
        print(f"✅ Retrieved {len(indicators)} indicators")
        sample_code = indicators[0]["code"]
        data = fetch_indicator_data(sample_code)
        print(f"📊 Sample indicator ({sample_code}) returned {len(data.get('series', []))} records")

    goals = fetch_goals()
    if goals:
        print(f"🌍 Retrieved {len(goals)} goals")

if __name__ == "__main__":
    test_all()


# from src.python.api_fetch import fetch_indicators, fetch_indicator_data

st.header("📊 UN SDG Data Explorer")

indicators = fetch_indicators()
if indicators:
    indicator_options = {ind["code"]: ind["title"] for ind in indicators}
    selected = st.selectbox("Choose an SDG Indicator", options=indicator_options.keys(), format_func=lambda k: indicator_options[k])
    data = fetch_indicator_data(selected)

    if data:
        st.subheader(f"Data for {indicator_options[selected]}")
        st.json(data)  # Customize with table or charts as needed
else:
    st.error("Failed to fetch indicators.")




